# Resolução - TP1

**Data:** 2025-10-14 17:56


Este notebook contém os **enunciados** e **gabaritos** dos 12 exercícios:
1-2 (TXT), 3-4 (JSON), 5-6 (sets), e 7-12 (Pandas com o dataset **Titanic**).



## 1. Nomes e tipos sanguíneos

**Enunciado**  
**a)** Implemente `gerar_arquivo_sangues(nomes, arquivo="sangues.txt")` que grave uma linha por pessoa no formato `nome,tipo`, sorteando **para cada nome** um tipo sanguíneo da lista `["A+","A-","B+","B-","AB+","AB-","O+","O-"]` **com repetição permitida**. Use UTF-8 e finalize cada registro com quebra de linha.  
Use esta lista de entrada:  
`["Ana Souza","Bruno Lima","Carla Nunes","Diego Alves","Elisa Martins","Fábio Rocha","Gabriela Reis","Heitor Cunha","Isabela Prado","João Silva","Larissa Moraes","Marcos Teixeira","Nina Ramos","Otávio Lopes","Paula Castro"]`

**b)** Adicione ao mesmo arquivo a linha **`Zilda Pereira,AB-`** e **salve no mesmo arquivo**.


In [ ]:

# Gabarito 1 (TXT)
from random import choice

TIPOS_SANGUINEOS = ["A+","A-","B+","B-","AB+","AB-","O+","O-"]

def gerar_arquivo_sangues(nomes, arquivo="sangues.txt"):
    with open(arquivo, "w", encoding="utf-8", newline="") as f:
        for nome in nomes:
            tipo = choice(TIPOS_SANGUINEOS)
            f.write(f"{nome},{tipo}\n")

# Lista fornecida no enunciado
nomes = ["Ana Souza","Bruno Lima","Carla Nunes","Diego Alves","Elisa Martins","Fábio Rocha",
         "Gabriela Reis","Heitor Cunha","Isabela Prado","João Silva","Larissa Moraes",
         "Marcos Teixeira","Nina Ramos","Otávio Lopes","Paula Castro"]

# (a) gerar arquivo
gerar_arquivo_sangues(nomes, arquivo="/mnt/data/sangues.txt")

# (b) adicionar a linha solicitada
with open("/mnt/data/sangues.txt", "a", encoding="utf-8", newline="") as f:
    f.write("Zilda Pereira,AB-\n")



## 2. Contagem por tipo sanguíneo

**Enunciado**  
**a)** Implemente `contar_sangues(arquivo="sangues.txt")` que leia o arquivo do exercício 1 e **retorne um dicionário `{tipo: contagem}`**.  
Ex.: pode retornar algo como `{"A+": 4, "O-": 2, "AB-": 1, ...}`.

**b)** Com base nesse dicionário, **qual é o tipo sanguíneo mais comum?**


In [ ]:

# Gabarito 2 (TXT)
VALIDOS = set(["A+","A-","B+","B-","AB+","AB-","O+","O-"])

def contar_sangues(arquivo="/mnt/data/sangues.txt"):
    cont = {}
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if not linha or "," not in linha:
                continue
            nome, tipo = linha.rsplit(",", 1)
            tipo = tipo.strip().upper()
            if tipo in VALIDOS:
                cont[tipo] = cont.get(tipo, 0) + 1
    return cont

contagens = contar_sangues("/mnt/data/sangues.txt")
contagens

# (b) Tipo mais comum
mais_comum = max(contagens.items(), key=lambda kv: kv[1]) if contagens else None
mais_comum



## 3. Criar e salvar JSON (json.dump)

**Enunciado**  
Implemente uma função `criar_perfil(limpar)` que leia do usuário: **nome** (texto), **idade** (inteiro), **renda** (float), **interesses** (string separada por vírgulas → lista sem duplicatas/espaços) e **ativo** (S/N → booleano). Monte um dicionário com esses campos (aplicando `limpar` nas strings) e **salve em `perfil.json`** usando `indent=4`.


In [ ]:

# Gabarito 3 (JSON - criação)
import json

def limpar_basico(s: str) -> str:
    # Remoção de espaços extras e caracteres simples indesejados nos extremos
    return " ".join(s.strip().split())

def criar_perfil(limpar=limpar_basico, path="/mnt/data/perfil.json", usar_inputs=False):
    if usar_inputs:
        nome = limpar(input("Nome: "))
        idade = int(input("Idade (int): "))
        renda = float(input("Renda (float): "))
        interesses_raw = input("Interesses (separados por vírgula): ")
        ativo_raw = input("Ativo? (S/N): ")
    else:
        # Exemplo de preenchimento programático (não interativo) para evitar bloqueio do notebook
        nome = limpar("Luis da Silva")
        idade = int(35)
        renda = float(7500.0)
        interesses_raw = "python, dados , ML, python"
        ativo_raw = "S"

    # Processamento
    interesses = [limpar(i).lower() for i in interesses_raw.split(",") if i.strip()]
    interesses = sorted(set(interesses))

    ativo = True if str(ativo_raw).strip().lower() in ("s", "sim", "true") else False

    perfil = {
        "nome": nome,
        "idade": idade,
        "renda": renda,
        "interesses": interesses,
        "ativo": ativo
    }

    with open(path, "w", encoding="utf-8") as f:
        json.dump(perfil, f, indent=4)

    return perfil

# Criar um perfil exemplo (não interativo) e salvar
perfil_demo = criar_perfil(usar_inputs=False)
perfil_demo



## 4. Ler/consultar e atualizar JSON (json.load + json.dump)

**Enunciado**  
Abra `perfil.json` (do ex. anterior) e peça ao usuário uma chave. Se a chave existir no perfil, mostre o valor (se for lista, mostre os itens em ordem alfabética). **Se a chave não existir**, peça um valor e **converta assim**: se contiver vírgulas → lista de strings únicas normalizadas; se for numérico, converta para **float**; se for `"s"`/`"n"` ou `"true"`/`"false"` (case-insensitive), **converta para booleano**; senão **mantenha string**. Adicione ao dicionário e **salve** em `perfil.json` com `indent=4`.


In [ ]:

# Gabarito 4 (JSON - leitura/atualização)
import json

def normalizar_lista(texto: str):
    itens = [i.strip() for i in texto.split(",") if i.strip()]
    # normalizar para minúsculas e remover duplicatas preservando ordem
    vistos = set()
    out = []
    for it in itens:
        low = it.lower()
        if low not in vistos:
            vistos.add(low)
            out.append(low)
    return out

def converter_valor(texto: str):
    t = texto.strip()
    if "," in t:
        return normalizar_lista(t)
    # numérico → float (tentativa)
    try:
        return float(t)
    except ValueError:
        pass
    # booleano simples
    low = t.lower()
    if low in ("s", "sim", "true"):
        return True
    if low in ("n", "nao", "não", "false"):
        return False
    return texto  # string original

def consultar_ou_criar_chave(path="/mnt/data/perfil.json", chave=None, valor_para_criar=None, usar_inputs=False):
    with open(path, "r", encoding="utf-8") as f:
        perfil = json.load(f)

    if usar_inputs:
        chave = input("Chave a consultar/criar: ").strip()
    if chave in perfil:
        valor = perfil[chave]
        if isinstance(valor, list):
            print(sorted(valor))
        else:
            print(valor)
    else:
        if usar_inputs:
            valor_para_criar = input("Valor para nova chave (texto): ")
        novo_valor = converter_valor(valor_para_criar if valor_para_criar is not None else "")
        perfil[chave] = novo_valor
        with open(path, "w", encoding="utf-8") as f:
            json.dump(perfil, f, indent=4)
        print(f"Chave '{chave}' criada.")

    return perfil

# Exemplo programático (cria a chave se não existir)
perfil_atualizado = consultar_ou_criar_chave(chave="apelido", valor_para_criar="Lu", usar_inputs=False)
perfil_atualizado.get("apelido")



## 5. Limpeza de e-mails com duplicatas (sets)

**Enunciado**  
Receba uma lista de e-mails com variações de maiúsculas/minúsculas e com **caracteres indesejados ou espaços** (ex.: `["  Ana.Silva@EXEMPLO.com  ", "ana.silva@example.com", "JOAO@Email.com", "joao@email.com ", "maria@site.com", "MARIA@SITE.COM"]`). Implemente uma função de limpeza que normalize cada e-mail (removendo caracteres indesejados/espaços e padronizando para minúsculas), use `set` para remover duplicatas e retorne a lista de e-mails únicos em ordem alfabética e o número de duplicatas removidas.


In [ ]:

# Gabarito 5 (Sets - e-mails)
import re

def limpar_email(s: str) -> str:
    # remove espaços nas bordas e caracteres invisíveis, normaliza para minúsculas
    s = s.strip().lower()
    # opcional: remover espaços internos esparsos
    s = re.sub(r"\s+", "", s)
    return s

def deduplicar_emails(emails):
    norm = [limpar_email(e) for e in emails if isinstance(e, str)]
    unicos = sorted(set(norm))
    removidas = len(norm) - len(unicos)
    return unicos, removidas

emails_ex = ["  Ana.Silva@EXEMPLO.com  ", "ana.silva@example.com", "JOAO@Email.com",
             "joao@email.com ", "maria@site.com", "MARIA@SITE.COM"]
deduplicar_emails(emails_ex)



## 6. Registros de alunos por CPF (sets)

**Enunciado**  
Dada uma lista de dicionários `{nome, cpf}` (ex.: `[{"nome": "Ana", "cpf": "111.111.111-11"}, {"nome": "Ana Maria", "cpf": "111.111.111-11"}, {"nome": "João", "cpf": "222.222.222-22"}, {"nome": "JOÃO", "cpf": "222.222.222-22"}, {"nome": "Carla", "cpf": "333.333.333-33"}]`), implemente uma função de limpeza para padronizar campos (ex.: remover caracteres indesejados/espaços e ajustar maiúsculas/minúsculas conforme julgar adequado), utilize um `set` para rastrear CPFs já vistos, **preserve a ordem** mantendo o primeiro registro de cada CPF e retorne a lista limpa e sem repetidos e o total de duplicatas descartadas.


In [ ]:

# Gabarito 6 (Sets - CPFs)
def limpar_nome(s: str) -> str:
    s = " ".join(s.strip().split())
    # Ex.: padronizar capitalização simples (não perfeito para todos os nomes)
    return " ".join([p.capitalize() for p in s.split(" ")])

def deduplicar_por_cpf(registros):
    vistos = set()
    limpo = []
    for a in registros:
        nome = a.get("nome", "")
        cpf = a.get("cpf", "")
        if cpf not in vistos:
            vistos.add(cpf)
            limpo.append({"nome": limpar_nome(nome), "cpf": cpf})
    descartadas = len(registros) - len(limpo)
    return limpo, descartadas

alunos_ex = [
    {"nome": "Ana",        "cpf": "111.111.111-11"},
    {"nome": "Ana Maria",  "cpf": "111.111.111-11"},
    {"nome": "João",       "cpf": "222.222.222-22"},
    {"nome": "JOÃO",       "cpf": "222.222.222-22"},
    {"nome": "Carla",      "cpf": "333.333.333-33"},
]
deduplicar_por_cpf(alunos_ex)



# Parte Titanic (Pandas)

Os exercícios abaixo referem-se ao dataset **Titanic**, usando Pandas.  
Você pode carregar com `sns.load_dataset("titanic")`, ao importar `seaborn as sns`, ou consultar o CSV em:  
https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv


In [ ]:

# Setup Titanic
import pandas as pd
import seaborn as sns

df = sns.load_dataset("titanic")
df.head()



## 7. Quem eram os passageiros mais ricos?
**a)** Liste os passageiros com `fare > 250`.  
**b)** Quantos estavam na 1ª classe?  
**c)** Quantos sobreviveram?


In [ ]:

# Gabarito 7
ricos = df.loc[df["fare"] > 250]
display(ricos[["survived", "pclass", "sex", "age", "fare", "embarked"]].sort_values("fare", ascending=False))

n_primeira = (ricos["pclass"] == 1).sum()
n_sobreviveram = (ricos["survived"] == 1).sum()

print("Qtd > 250:", len(ricos))
print("Na 1ª classe:", n_primeira)
print("Sobreviveram:", n_sobreviveram)



## 8. Idade média por classe
Calcule a idade média por classe (`pclass`), ordenando da 1ª para a 3ª classe.


In [ ]:

# Gabarito 8
m = (df.groupby("pclass")["age"]
       .mean(numeric_only=True)
       .sort_index())
m



## 9. Taxa de sobrevivência por sexo e classe
Monte uma **pivot table** com a média de sobreviventes por `sexo` (linhas) e `pclass` (colunas).


In [ ]:

# Gabarito 9
pt = pd.pivot_table(
    df,
    values="survived",
    index="sex",
    columns="pclass",
    aggfunc="mean"
).sort_index(axis=1)
pt



## 10. Faixas de idade e sobrevivência
Crie faixas de idade: `[0,12)`, `[12,18)`, `[18,35)`, `[35,60)`, `60+`.  
Calcule a taxa de sobrevivência por faixa e por sexo.


In [ ]:

# Gabarito 10
bins = [0, 12, 18, 35, 60, float("inf")]
labels = ["criança", "adolescente", "jovem", "adulto", "idoso"]
df2 = df.copy()
df2["faixa_idade"] = pd.cut(df2["age"], bins=bins, labels=labels, right=False)

taxa = (df2
        .groupby(["faixa_idade", "sex"])["survived"]
        .mean()
        .reset_index()
        .sort_values(["faixa_idade", "sex"]))
taxa



## 11. Família a bordo e sobrevivência
Considere que viajou com família quem tem `sibsp + parch > 0`.  
**a)** Crie a coluna booleana `com_familia`.  
**b)** Compare a taxa de sobrevivência entre `com_familia=True` e `False`, por classe.


In [ ]:

# Gabarito 11
df3 = df.copy()
df3["com_familia"] = (df3["sibsp"].fillna(0) + df3["parch"].fillna(0)) > 0

res = (df3
       .groupby(["pclass", "com_familia"])["survived"]
       .mean()
       .reset_index()
       .sort_values(["pclass", "com_familia"]))
res



## 12. Portos e ranking por tarifa
**a)** Crie um mapeamento `embarked → porto` (`C→Cherbourg`, `Q→Queenstown`, `S→Southampton`) e adicione a coluna `porto`.  
**b)** Para cada `porto` e `pclass`, liste o **top-3** maiores valores de `fare`, mantendo o índice original.


In [ ]:

# Gabarito 12
mapa = pd.DataFrame({
    "embarked": ["C", "Q", "S"],
    "porto": ["Cherbourg", "Queenstown", "Southampton"]
})

dfm = df.merge(mapa, on="embarked", how="left")

top3 = (dfm
        .sort_values("fare", ascending=False)
        .groupby(["porto", "pclass"], group_keys=False)
        .head(3)
        .loc[:, ["porto", "pclass", "fare", "sex", "age", "survived"]])

top3
